In [20]:
%pwd

'C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops'

In [21]:
import os
from pathlib import Path
os.chdir(Path("C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops\\"))

In [22]:
%pwd

'C:\\Users\\kural\\Desktop\\Projects\\End_To_End_MLops'

In [23]:
# from software_defect_prediction.constants import *
from software_defect_prediction.utils.common import *
# from software_defect_prediction.entity.config_entity import DataIngestionConfig
# from software_defect_prediction.config.configuration import ConfigurationManager

In [24]:
from dataclasses import dataclass
from pathlib import Path
from software_defect_prediction import logger

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    source_URL_file_name : str
    source_URL_unzip_file_name : str
    local_data_file: Path
    unzip_dir: Path

In [25]:
from software_defect_prediction.constants import *
from software_defect_prediction.utils.common import read_yaml, create_directories
from software_defect_prediction.entity.config_entity import (DataIngestionConfig)

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            source_URL_file_name = config.source_URL_file_name,
            source_URL_unzip_file_name = config.source_URL_unzip_file_name,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [26]:
from software_defect_prediction.entity.config_entity import DataIngestionConfig
from software_defect_prediction import logger

import urllib.request
from pathlib import Path
import zipfile

 
class Data_Ingestion():
    def __init__(self,data_ingestion_config : DataIngestionConfig) -> None:
        self.config = data_ingestion_config
    
    def download_file(self) -> None:
        save_path = Path(self.config.root_dir,self.config.source_URL_file_name)
        try:
            # Open a connection to the URL
            with urllib.request.urlopen(self.config.source_URL) as response:
                # Read data from the response
                data = response.read()
                
                # Write the data to a file at the specified save path
                with open(save_path, 'wb') as file:
                    file.write(data)
                    
            logger.info(f"File downloaded successfully to: {save_path}")
        except Exception as e:
            logger.error(f"Failed to download file from {self.config.source_URL}: {e}")
            
    def unzip_file(self) -> None:
        
        zip_file_path = Path(self.config.root_dir,self.config.source_URL_file_name) 
        extract_dir = self.config.unzip_dir
        from_rename = self.config.source_URL_unzip_file_name
        to_rename = self.config.local_data_file
        
        try:
            # Create the extraction directory if it doesn't exist
            os.makedirs(extract_dir, exist_ok=True)

            # Open the zip file for reading
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                # Extract all contents of the zip file to the extraction directory
                zip_ref.extractall(extract_dir)
                os.replace(Path(extract_dir,from_rename),Path(extract_dir,to_rename))

            logger.info(f"Successfully extracted {zip_file_path} to {extract_dir}")
        except Exception as e:
            logger.error(f"Failed to unzip {zip_file_path}: {e}")

In [27]:
configuration_obj = ConfigurationManager()
data_ingestion_conf =  configuration_obj.get_data_ingestion_config()
step_data_ing = Data_Ingestion(data_ingestion_conf)
step_data_ing.download_file()
step_data_ing.unzip_file()
logger.success("Data ingestion completed successfully")

2024-05-24 09:54:12.331 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: config\config.yaml loaded successfully
2024-05-24 09:54:12.334 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: params.yaml loaded successfully
2024-05-24 09:54:12.339 | INFO     | software_defect_prediction.utils.common:read_yaml:31 - yaml file: schema.yaml loaded successfully
2024-05-24 09:54:12.342 | INFO     | software_defect_prediction.utils.common:create_directories:51 - created directory at: artifacts
2024-05-24 09:54:12.344 | INFO     | software_defect_prediction.utils.common:create_directories:51 - created directory at: artifacts/data_ingestion
2024-05-24 09:54:16.141 | INFO     | __main__:download_file:25 - File downloaded successfully to: artifacts\data_ingestion\software_defect_detection.zip
2024-05-24 09:54:16.239 | INFO     | __main__:unzip_file:46 - Successfully extracted artifacts\data_ingestion\software_defect_detection.zip to artifacts/d

In [28]:
data_ingestion_conf.root_dir

'artifacts/data_ingestion'